In [2]:
import pandas as pd
import pingouin as pg

df = pd.read_csv('concatenated_df.csv')

#df.drop(['Unnamed: 0', 'location'], axis=1, inplace=True)

variables_of_interest = list(df.columns)

partial_corr_result = df[variables_of_interest].pcorr()

partial_corr_fwix = partial_corr_result['fwix'].drop('fwix')

partial_corr_fwix_rounded = partial_corr_fwix.round(3)

print(partial_corr_fwix_rounded)

index                                     NaN
Unnamed: 0                                NaN
latitude                                  NaN
longitude                                 NaN
elevation                                 NaN
hourly.temperature_2m                     NaN
hourly.relative_humidity_2m               NaN
hourly.dew_point_2m                       NaN
hourly.apparent_temperature               NaN
hourly.precipitation                      NaN
hourly.rain                               NaN
hourly.snowfall                           NaN
hourly.snow_depth                         NaN
hourly.weather_code                       NaN
hourly.pressure_msl                       NaN
hourly.surface_pressure                   NaN
hourly.cloud_cover                        NaN
hourly.cloud_cover_low                    NaN
hourly.cloud_cover_mid                    NaN
hourly.cloud_cover_high                   NaN
hourly.et0_fao_evapotranspiration         NaN
hourly.vapour_pressure_deficit    

In [1]:
print(df.head())

NameError: name 'df' is not defined

In [7]:
import numpy as np

In [12]:
grouped = df.groupby('location')

partial_corrs = [] 

for name, group in grouped:
    group = group.drop(['index.1', 'Unnamed: 0', 'location', 'index','latitude','hourly.is_day','longitude','elevation','hourly.snowfall', 'hourly.snow_depth', 'hourly.weather_code'], axis=1)

    # Get the last 24*8 rows of each group
    df_last_rows = group.tail(24*30)

    # Check if there are enough rows
    if df_last_rows.shape[0] < 24*30:
        print(f"Location: {name} has insufficient data")
        continue

    # Check for missing or infinite values in numeric columns
    numeric_cols = df_last_rows.select_dtypes(include=[np.number]).columns
    if df_last_rows[numeric_cols].isnull().any().any() or np.isinf(df_last_rows[numeric_cols]).any().any():
        print(f"Location: {name} has missing or infinite values")
        continue

    # Check for lack of variation
    if df_last_rows.nunique(dropna=False).min() <= 1:
        print(f"Location: {name} has a variable with no variation")
        continue

    # Now, you can calculate the partial correlation on this new dataframe
    variables_of_interest = list(df_last_rows.columns)

    partial_corr_result = df_last_rows[variables_of_interest].pcorr()

    partial_corr_fwix = partial_corr_result['fwix'].drop('fwix')

    partial_corr_fwix_rounded = partial_corr_fwix.round(3)

    partial_corrs.append(partial_corr_fwix_rounded)

    #print(f"Location: {name}")
    #print(partial_corr_fwix_rounded)

average_partial_corr = pd.concat(partial_corrs, axis=1).mean(axis=1)

print(average_partial_corr)

Location: 240 has a variable with no variation
Location: 338 has a variable with no variation
Location: 339 has a variable with no variation
Location: 340 has a variable with no variation
Location: 346 has a variable with no variation
Location: 416 has a variable with no variation
Location: 552 has a variable with no variation
Location: 793 has a variable with no variation
Location: 794 has a variable with no variation
Location: 836 has a variable with no variation
Location: 842 has a variable with no variation
Location: 868 has a variable with no variation
Location: 869 has a variable with no variation
Location: 870 has a variable with no variation
Location: 879 has a variable with no variation
Location: 1150 has a variable with no variation
Location: 1151 has a variable with no variation
Location: 1156 has a variable with no variation
Location: 1185 has a variable with no variation
Location: 1186 has a variable with no variation
Location: 1188 has a variable with no variation
Locatio

In [18]:
grouped = df.groupby('location')

partial_corrs = [] 

for name, group in grouped:
    group = group.drop(['index.1', 'Unnamed: 0', 'location', 'index','latitude','hourly.is_day','longitude','elevation','hourly.snowfall', 'hourly.snow_depth', 'hourly.weather_code'], axis=1)

    # Convert 'hourly.time' to datetime and set it as the index
    group['hourly.time'] = pd.to_datetime(group['hourly.time'])
    group.set_index('hourly.time', inplace=True)
    group.sort_index(inplace=True)

    # Get the last 30 days of each group
    df_last_rows = group.last('8D')

    # Check if there are enough days
    if len(df_last_rows.resample('D')) < 8:
        print(f"Location: {name} has insufficient data")
        continue

    # Calculate the daily average
    df_last_rows = df_last_rows.resample('D').mean()

    # Check for missing or infinite values in numeric columns
    numeric_cols = df_last_rows.select_dtypes(include=[np.number]).columns
    if df_last_rows[numeric_cols].isnull().any().any() or np.isinf(df_last_rows[numeric_cols]).any().any():
        print(f"Location: {name} has missing or infinite values")
        continue

    # Check for lack of variation
    if df_last_rows.nunique(dropna=False).min() <= 1:
        print(f"Location: {name} has a variable with no variation")
        continue

    # Now, you can calculate the partial correlation on this new dataframe
    variables_of_interest = list(df_last_rows.columns)

    partial_corr_result = df_last_rows[variables_of_interest].pcorr()

    partial_corr_fwix = partial_corr_result['fwix'].drop('fwix')

    partial_corr_fwix_rounded = partial_corr_fwix.round(3)

    partial_corrs.append(partial_corr_fwix_rounded)

average_partial_corr = pd.concat(partial_corrs, axis=1).mean(axis=1)

print(average_partial_corr)

/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('

Location: 2 has a variable with no variation
Location: 4 has a variable with no variation
Location: 13 has a variable with no variation


/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('

Location: 58 has a variable with no variation
Location: 63 has a variable with no variation
Location: 66 has a variable with no variation


/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('

...

Location: 1727 has a variable with no variation
Location: 1728 has a variable with no variation
Location: 1729 has a variable with no variation


/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('

Location: 1752 has a variable with no variation


/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('8D')
/tmp/ipykernel_36857/4212870766.py:14: FutureWarning: last is deprecated and will be removed in a future version. Please create a mask and filter using `.loc` instead
  df_last_rows = group.last('

Location: 1769 has a variable with no variation
hourly.temperature_2m                     -0.214270
hourly.relative_humidity_2m                0.565081
hourly.dew_point_2m                        0.378422
hourly.apparent_temperature                0.016230
hourly.precipitation                       0.146008
hourly.rain                                0.146008
hourly.pressure_msl                        0.017342
hourly.surface_pressure                   -0.006467
hourly.cloud_cover                         0.019174
hourly.cloud_cover_low                     0.039232
hourly.cloud_cover_mid                    -0.030748
hourly.cloud_cover_high                    0.056234
hourly.et0_fao_evapotranspiration         -0.614616
hourly.vapour_pressure_deficit            -0.458082
hourly.wind_speed_10m                     -0.383654
hourly.wind_speed_100m                    -0.378295
hourly.wind_direction_10m                 -0.043548
hourly.wind_direction_100m                -0.037086
hourly.wind_gust